In [1]:
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle

# Feature extraction using VGG16
def extract_features(image_path):
    # Load the VGG16 model pre-trained on ImageNet
    base_model = VGG16(weights="imagenet")
    model = Model(inputs=base_model.input, outputs=base_model.layers[-2].output)

    # Preprocess the image
    image = load_img(image_path, target_size=(224, 224))  # Resize image
    image = img_to_array(image)  # Convert to numpy array
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    image = preprocess_input(image)  # Normalize the image

    # Extract features
    features = model.predict(image, verbose=0)
    return features

# Load tokenizer and pre-trained caption model
tokenizer = pickle.load(open('tokenizer.pkl', 'rb'))  # Pre-trained tokenizer
caption_model = load_model('caption_model.h5')        # Trained captioning model

# Caption generation function
def generate_caption(features, max_length):
    in_text = 'startseq'
    for _ in range(max_length):
        # Convert current sequence to integers
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)

        # Predict the next word
        y_pred = caption_model.predict([features, sequence], verbose=0)
        y_pred = np.argmax(y_pred)  # Get the word with the highest probability

        # Map integer back to word
        word = tokenizer.index_word.get(y_pred)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'endseq':
            break
    return in_text.replace('startseq', '').replace('endseq', '').strip()

# Main function
if __name__ == "__main__":
    # Set image path
    image_path = 'sample_image.jpg'  # Replace with your test image path

    # Extract image features
    features = extract_features(image_path)

    # Generate caption
    max_length = 34  # Adjust based on your dataset
    caption = generate_caption(features, max_length)

    print("Generated Caption:", caption)


FileNotFoundError: [Errno 2] No such file or directory: 'tokenizer.pkl'